In [21]:
from plot_log import load_logs
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
load_logs('examples/sofas', "loss")

In [ ]:
load_logs('examples/sofas', "learning_rate")

In [ ]:
load_logs('examples/sofas', "time")

In [ ]:
load_logs('examples/sofas', "lat_mag")

In [22]:
from pyntcloud import PyntCloud

vis_path = "examples/sofas/Reconstructions/100/Meshes/ShapeNetV2/04256520/d4c6f2749bd85e9a6a9633583f89b17f.ply"
cloud = PyntCloud.from_file(vis_path)

In [23]:
converted = cloud.get_sample("mesh_random", n=100000, as_PyntCloud=True)

In [24]:
converted.plot(background='white', initial_point_size=0.008)

Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.006840962916612625, 0.28448450565338135, 0…